In [4]:
from gridworld import *
from tqdm import trange
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#Normal Q-learning


gw = gridworld((4, 4), (0, 3), (3, 3), [(o, 3) for o in range(1, 3)], 0.9, 0, 100, 0)

In [5]:
def dist_error_state(state, error_state):
    return ( np.sqrt( (state[0] - error_state[0])**2 + (state[1] - error_state[1])**2 ) )

risk_states = np.zeros((gw.M, gw.N))
for i in range(gw.M):
    for j in range(gw.N):
#         if (i, j) in gw.error_states:
#             risk_states[i,j] == 0.
#         if (i, j) == gw.goal:
#             risk_states[i,j] == 0.
#         else:
        sm = 100.
        for e in gw.error_states:
            #sm += np.exp( - dist_error_state([i,j], e) )
            sm = min(sm, dist_error_state([i,j], e))
        risk_states[i,j] = sm
risk_states
np.random.seed(1)

np.min(risk_states)
np.max(risk_states)
risk_states = (risk_states - np.min(risk_states))/(np.max(risk_states) - np.min(risk_states)) * 5
risk_states

array([[3.16227766, 2.23606798, 1.41421356, 1.        ],
       [3.        , 2.        , 1.        , 0.        ],
       [3.        , 2.        , 1.        , 0.        ],
       [3.16227766, 2.23606798, 1.41421356, 1.        ]])

0.0

3.1622776601683795

array([[5.        , 3.53553391, 2.23606798, 1.58113883],
       [4.74341649, 3.16227766, 1.58113883, 0.        ],
       [4.74341649, 3.16227766, 1.58113883, 0.        ],
       [5.        , 3.53553391, 2.23606798, 1.58113883]])

In [6]:
#Empirically estimate safety
num_sims = 10000
def estimate_rr(pi):
    rewards = []
    discounted = []
    risks = []
    discounted_reward = 0
    for i in range(num_sims):
        s = gw.reset()
        reward = 0
        d = False
        j = 0
        risk_dist = 0
        while(j < 1000):
            a = pi[s[0],s[1]]
            s1,r,d = gw.step(a)
            reward += r
            risk_dist += risk_states[s[0], s[1]]                     
            if d == True:
                if s1 in gw.error_states:
                    discounted_reward = 0
                else:
                    discounted_reward = pow(y, j) * r
                break
            s = s1
            j += 1
        rewards.append(reward)
        discounted.append(discounted_reward)
        risks.append(risk_dist)
    return sum(discounted)/float(num_sims), sum(rewards)/float(num_sims), sum(risks)/float(num_sims)  

In [7]:
trials = 3
Q_list = []
reward_trial_list = []

np.random.seed(1)

# Set learning parameters
lr = 0.9
num_episodes = 10000

for k in range(trials):
    #Initialize table with all zeros
    Q = np.zeros((gw.M,gw.N,4))
    jList = []
    rList = []
    for i in trange(num_episodes):
        #print('Episode', i)
        #Reset environment and get first new observation
        s = gw.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < 1000:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            a = np.argmax(Q[s[0],s[1],:] + np.random.randn(1,4)*(1./(i+1)))
#             if np.random.rand() < epsilon:
#                 # take random action
#                 a = np.random.choice([0,1,2,3])
#             else:
#                 # take action according to the q function table
#                 a = np.argmax(Q[s[0],s[1],:])
            #Get new state and reward from environment
            s1,r,d = gw.step(a)

            #Update Q-Table with new knowledge
            Q[s[0], s[1],a] = Q[s[0], s[1],a] + lr*(r + y*np.max(Q[s1[0], s1[1],:]) - Q[s[0], s[1],a])

            rAll += r
            s = s1
            if d == True:
                break
        jList.append(j)    
        rList.append(rAll)
        
    Q_list.append(Q)
#     v = np.mean(Q, axis = 2)
#     gw.print_v(v)

    pi = np.argmax(Q, axis = 2)
    gw.print_policy(pi)
#     reward_trial_list.append(sum(rList)/float(num_episodes))
#     print "Score over time: " +  str(sum(rList)/float(num_episodes))

#     window = int(num_episodes/10)

#     plt.figure(figsize=[9,16])
#     plt.subplot(411)
#     plt.plot(pd.Series(jList).rolling(window).mean())
#     plt.title('Step Moving Average ({}-episode window)'.format(window))
#     plt.ylabel('Moves')
#     plt.xlabel('Episode')

#     plt.subplot(412)
#     plt.plot(pd.Series(rList).rolling(window).mean())
#     plt.title('Reward Moving Average ({}-episode window)'.format(window))
#     plt.ylabel('Reward')
#     plt.xlabel('Episode')

#     plt.tight_layout(pad=2)
#     plt.show()

gw.print_gw()
meanQ = np.mean(np.array(Q_list), axis = 0)
meanv = np.mean(meanQ, axis = 2)
gw.print_v(meanv)
meanpi = np.argmax(meanQ, axis = 2)
gw.print_policy(meanpi)
E_dis_reward, E_reward, E_risk = estimate_rr(meanpi)
print "Mean reward (empirical)", E_dis_reward
print "Successful Episodes ", E_reward
print "Safety ", E_risk

100%|██████████| 10000/10000 [00:02<00:00, 4939.11it/s]


,0,1,2,3
0,⬇,⬇,⬇,⬅
1,⬇,⬇,⬅,E
2,➡,⬇,⬅,E
3,⬆,➡,➡,G


100%|██████████| 10000/10000 [00:02<00:00, 3544.93it/s]


,0,1,2,3
0,⬇,⬅,⬅,⬅
1,⬇,⬆,⬇,E
2,⬇,➡,⬇,E
3,➡,➡,➡,G


100%|██████████| 10000/10000 [00:01<00:00, 6747.76it/s]


,0,1,2,3
0,➡,➡,⬇,⬅
1,⬇,➡,⬇,E
2,➡,➡,⬇,E
3,➡,⬆,➡,G


,0,1,2,3
0,-,-,-,S
1,-,-,-,E
2,-,-,-,E
3,-,-,-,G


,0,1,2,3
0,14.54,15.38,19.41,18
1,15.80,19.13,20.20,E
2,19.75,21.23,21.52,E
3,19.21,22.07,25.00,G


,0,1,2,3
0,⬇,⬅,⬇,⬅
1,⬇,⬇,⬇,E
2,➡,➡,⬇,E
3,➡,➡,➡,G


Mean reward (empirical) 69.9906138205
Successful Episodes  89.41
Risk probab  10.836533508745259
